In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm_notebook

from fastmri_recon.data.fastmri_sequences import ZeroFilled2DSequence
from fastmri_recon.helpers.adversarial_training import adversarial_training_loop
from fastmri_recon.helpers.image_tboard_cback import TensorBoardImage
from fastmri_recon.helpers.keras_utils import wasserstein_loss
from fastmri_recon.models.discriminator import discriminator_model
from fastmri_recon.models.unet import unet

Using TensorFlow backend.


In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

In [3]:
run_params = {
    'n_layers': 4, 
    'pool': 'max', 
    "layers_n_channels": [16, 32, 64, 128], 
    'layers_n_non_lins': 2,
}

def generator_model():
    model = unet(input_size=(320, 320, 1), **run_params, compile=False)
    return model

In [4]:
# model definitions
g = generator_model()
# prev_run_id = 'unet_wo_lastrelu_af4_1572013433'
# prev_epoch = 500
# chkpt_path = f'checkpoints/{prev_run_id}-{prev_epoch}.hdf5'
# g.load_weights(chkpt_path)
d = discriminator_model()

In [5]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [6]:
# model compiling
perceptual_loss = 'mae'
discriminator_lr = 1e-4
reconstructor_lr = 1e-3
perceptual_weight = 1.0

In [7]:
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
n_volumes_train = 973
n_volumes_val = 199
AF = 4
train_gen = ZeroFilled2DSequence(train_path, af=AF, norm=True)
val_gen = ZeroFilled2DSequence(val_path, af=AF, norm=True)

epoch_num = 100
critic_updates = 5

run_id = f'unet_gan_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

unet_gan_af4_1578672239


In [8]:
log_dir = op.join('logs', run_id)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    write_graph=True, 
    update_freq=50,
)
chkpt_cback = ModelCheckpoint(chkpt_path, period=epoch_num)
selected_slice = 15
tboard_image_cback = TensorBoardImage(
    log_dir=log_dir + '/images',
    image=val_gen[0][1][selected_slice:selected_slice+1],
    # NOTE: for cross-domain slice has to be on kspace and mask
    model_input=val_gen[0][0][selected_slice:selected_slice+1],
)

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/data_utils.py:18: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(filename) as h5_obj:


In [9]:
# %%time
# adversarial_training_loop(
#     g, 
#     d, 
#     d_on_g, 
#     train_gen, 
#     val_gen=val_gen,
#     validation_steps=1,
#     n_epochs=epoch_num, 
#     n_batches=n_volumes_train, 
#     n_critic_updates=critic_updates,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, tboard_image_cback],
#     workers=35,
#     use_multiprocessing=True,
#     max_queue_size=35,
#     include_d_metrics=True,
# )

In [10]:
%%time
#overfitting trial
adversarial_training_loop(
    g, 
    d,
    train_gen, 
    d_lr=discriminator_lr,
    g_lr=reconstructor_lr,
#     val_gen=val_gen,
#     validation_steps=1,
    n_epochs=500, 
    n_batches=1, 
    n_critic_updates=5,
    callbacks=[tqdm_cb, tboard_cback, tboard_image_cback],
#     workers=35,
#     use_multiprocessing=True,
#     max_queue_size=35,
    include_d_metrics=False,
    perceptual_weight=perceptual_weight,
    perceptual_loss=perceptual_loss,
)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/data_utils.py:18: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(filename) as h5_obj:
/volatile/home/Z

InternalError:  Blas GEMM launch failed : a.shape=(36, 400), b.shape=(400, 1024), m=36, n=1024, k=400
	 [[node Discriminator/dense/MatMul (defined at /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/helpers/adversarial_training.py:91) ]] [Op:__inference_train_step_10141]

Function call stack:
train_step
